In [1]:
import torch
from torch.optim import AdamW
from testbed import TextDataset, Trainer, Net0, Net1, Transformer
from testbed.util import decode_broken_utf8, default_device

Loading BokehJS ...

In [2]:
K = 8 # token embedding dimension (K stands for "keys" but dont ask for justification)
C = 256 # number of tokens (C stands for "classes")

In [3]:
model_type = "Net0"

In [4]:
# Net0
#   * N = L + 1
if model_type == "Net0":
    H = 8192*2 # 2**10 # number of hidden neurons (i.e. number of convolution kernels)
    L = 128*2 # 128 # length of convolution kernel
    N = L + 1
    B = 8192 # batch size (i.e. examples per batch)
    model = Net0(H=H, L=L, K=K, C=C).to(default_device())
    dataset = TextDataset(N=N)
    optimizer = AdamW(model.parameters())

In [5]:
# Net 1
#   * N > L
if model_type == "Net1":
    H = 4096 # number of hidden neurons (i.e. number of convolution kernels)
    L = 1024 # length of convolution kernel
    N = 2*L
    B = 32 # batch size (i.e. examples per batch)
    model = Net1(H=H, L=L, K=K, C=C).to(default_device())
    dataset = TextDataset(N=N)
    optimizer = AdamW(model.parameters())

In [6]:
if model_type == "Transformer":
    N = 8
    model = Transformer().to(default_device())
    dataset = TextDataset(N=N)
    optimizer = AdamW(model.parameters())

In [7]:
model_size = sum([p.numel() for p in model.parameters()])
print(model_size)

37767424


In [8]:
len(dataset)

57347693

In [13]:
trainer = Trainer(model=model, N=N, B=B, dataset=dataset, optimizer=optimizer)

In [ ]:
trainer.load()

In [ ]:
trainer.losses = []
trainer.compute_time = 0.0

In [14]:
trainer.start()

In [15]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.plotting import figure 
output_notebook()
import time, math
import numpy as np
from threading import Thread, Lock

class StatsTicker:
    def __init__(self, trainer):
        self.trainer = trainer
        self.tick = 0
        self.losses = self.trainer.update_losses()
        self.bokeh = {}
        self.bokeh_handle = None
        self.updating = False
        
    def recent_stats(self):
        self.losses = self.trainer.update_losses()
        data = {'time' : [ x[1] for x in self.losses[self.tick:]],
                'mean_loss' : [8*x[2]/math.log(256) for x in self.losses[self.tick:]]}
        return data
    
    def display(self, updates=True):
        TOOLS="pan,wheel_zoom,box_zoom,reset"
        self.bokeh["figure"] = figure(tools=TOOLS)
        self.bokeh["figure"].axis.major_label_text_font_size = "24px"
        hover = HoverTool(show_arrow=False,
                          mode='vline',
                          line_policy='next',
                          tooltips=[('X_value', '$data_x'),
                                    ('Y_value', '$data_y')])
        self.bokeh["figure"].add_tools(hover)
        data = self.recent_stats()
        self.bokeh["mean_loss"] = self.bokeh["figure"].line(data['time'],data['mean_loss'])
        self.tick = len(self.losses)
        self.bokeh_handle = show(self.bokeh["figure"], notebook_handle=True)
        if updates:
            self.start()
            
    def start(self):
        if not self.updating:
            self.updating = True
            self.updater = Thread(target=StatsTicker._update_loop, args=(self,), daemon=True)
            self.updater.start()
            
    def stop(self):
        if self.updating:
            self.updating = False
            self.updater.join()
            
    def _update_loop(self):
        while self.updating:
            time.sleep(1)
            data = self.recent_stats()
            if len(self.losses) > self.tick:
                self.bokeh["mean_loss"].data_source.stream({'x':data['time'], 
                                                            'y':data['mean_loss']})
                self.tick = len(self.losses)
            push_notebook(handle=self.bokeh_handle)

Loading BokehJS ...

In [16]:
ticker = StatsTicker(trainer)

In [17]:
ticker.display()

In [25]:
trainer.set_batch_size(8192*3)

In [29]:
ticker.losses[-10:]

[[2696, 1219.5381014347076, 2.0433592001597085],
 [2699, 1221.128886461258, 2.0296196937561035],
 [2702, 1222.3952286243439, 2.0439936319986978],
 [2705, 1223.7032783031464, 2.0385449727376304],
 [2708, 1225.048589706421, 2.0329339504241943],
 [2711, 1226.6886582374573, 2.0577073097229004],
 [2714, 1227.9812579154968, 2.042966445287069],
 [2717, 1229.273854970932, 2.0342026551564536],
 [2720, 1230.560406446457, 2.03302264213562],
 [2723, 1232.1519467830658, 2.050114313761393]]

In [ ]:
trainer.save()

In [ ]:
def smoother(data, lag):
    cs = np.cumsum(np.array(data))
    return (cs[lag:] - cs[:-lag])/lag

In [ ]:
TOOLS="crosshair,pan,wheel_zoom,box_zoom,reset,tap,box_select,lasso_select"
logfig = figure(tools=TOOLS)
logfig.axis.major_label_text_font_size = "24px"
hover = HoverTool(tooltips=None, mode="vline")
logfig.add_tools(hover)
lag = 0
if lag > 0:
    X = [x[1] for x in trainer.losses][lag:]
    Y = [ 8*u / math.log(256) for u in smoother([x[2] for x in trainer.losses], lag)]
else:
    X = [x[1] for x in trainer.losses]
    Y = [x[2] for x in trainer.losses]
logline = logfig.line(X,Y)
bokeh_handle = show(logfig)

In [ ]:
trainer.autocomplete()
pass

In [ ]:
for p in trainer.model.parameters():
    print(p.shape, p, torch.max(p).item(), torch.min(p).item())

In [ ]:
trainer.stop()

In [ ]:
with torch.no_grad():
    for i in range(1000):
        print(model(dataset[i].view(1,-1).long().cuda()).item())